# Problem 2-1 and 2-2

1. calculate fraction of total flux from a number of blackbodies which falls within idealized Johnson filter passbands
2. calculate the B-V color indices of those blackbodies

In [16]:
import astropy
import numpy as np
import astropy.units as u
import astropy.constants as const
from functools import partial

temps = [10**5, 35000, 9700, 6500, 4700, 2600]*u.K

B = (3900,4900)*u.Angstrom
V = (5050, 5950)*u.Angstrom

filters = [B,V]

# stefan-Boltzmann formula for blackbody radiation
def stefan_boltzmann_formula(T):
    return const.sigma_sb * T**4

# planks law for blackbody radiation
def planks_law(T, lam):
    return 2*const.h * const.c**2 / (lam**5 * (np.exp(const.h * const.c / (lam * const.k_B * T)) - 1))

# integrate a function numerically, using Simpson's rule
def simpsons_rule(f, a, b):
    return (b - a) / 6 * (f(a) + 4 * f((a + b) / 2) + f(b))

def filter_flux(filter, temp):
    radiance = simpsons_rule(partial(planks_law, temp), filter[0], filter[1]).to(u.W/u.m**2)
    return radiance * np.pi

In [ ]:
#problem 2-1

for temp in temps:
    flux_bol = stefan_boltzmann_formula(temp)

    print(f"Temperature: {temp:.0f}, Bolometric Flux: {flux_bol:.2e}")

    spectral_radiance_5050=planks_law(temp, 5050*u.Angstrom).to(u.W/u.m**2/u.m)#u.Angstrom)
    print(f"Spectral radiance at 5050 Angstrom: {spectral_radiance_5050:.2e}")
    
    for filter in filters:
        # this form of Planck's law returns units of radiance
        # these must be integrated over all solid angles to get surface flux
        
        flux = filter_flux(filter, temp)
        
        percent = flux / flux_bol *100
        print(f"  Filter {filter}: {flux:.2e}, percent: {percent:.2f}%")


In [19]:
# problem 2-2

# the zero point magnitude for the B and V filters is at 9700 K
B_ref = filter_flux(B, 9700*u.K)
V_ref = filter_flux(V, 9700*u.K)

for temp in temps:
    B_flux = filter_flux(B, temp)
    V_flux = filter_flux(V, temp)

    B_mag = 2.5 * np.log10(B_ref / B_flux)
    V_mag = 2.5 * np.log10(V_ref / V_flux)

    CI = B_mag - V_mag

    print(f"Temperature: {temp:.0f}, B magnitude: {B_mag:.2f}, V magnitude: {V_mag:.2f}, CI: {CI:.2f}")


Temperature: 100000 K, B magnitude: -4.70, V magnitude: -4.18, CI: -0.51
Temperature: 35000 K, B magnitude: -3.18, V magnitude: -2.75, CI: -0.43
Temperature: 9700 K, B magnitude: 0.00, V magnitude: 0.00, CI: 0.00
Temperature: 6500 K, B magnitude: 1.85, V magnitude: 1.51, CI: 0.34
Temperature: 4700 K, B magnitude: 3.94, V magnitude: 3.20, CI: 0.74
Temperature: 2600 K, B magnitude: 9.95, V magnitude: 8.06, CI: 1.89
